In [1]:
from myplt import *
from sklearn.neighbors import KDTree
import h5py

In [2]:
for k,v in params.items():
    print(k,v)

NameError: name 'params' is not defined

In [77]:
f = h5py.File('./galaxy_collide/t1/output/snapshot_%0.3d.hdf5'%snapshot)
p = f['PartType1']
for k,v in p.items():
    print(k,v)

Coordinates <HDF5 dataset "Coordinates": shape (40000, 3), type "<f4">
ParticleIDs <HDF5 dataset "ParticleIDs": shape (40000,), type "<u4">
Velocities <HDF5 dataset "Velocities": shape (40000, 3), type "<f4">


In [76]:
f = h5py.File('./galaxy_collide/t1/output/snapshot_%0.3d.hdf5'%snapshot)
p = f['PartType2']
for k,v in p.items():
    print(k,v)

Coordinates <HDF5 dataset "Coordinates": shape (20000, 3), type "<f4">
ParticleIDs <HDF5 dataset "ParticleIDs": shape (20000,), type "<u4">
Velocities <HDF5 dataset "Velocities": shape (20000, 3), type "<f4">


In [87]:
x=np.array(p['Coordinates'])

In [92]:
np.min(x[:, 2])

-315.82767

In [105]:
def h5tocsv( snapshot, parttype, inprefix ,outprefix, npt=8, xselect=None ):
    f = h5py.File('%ssnapshot_%0.3d.hdf5'%(inprefix, snapshot))
    params = f['Header'].attrs
    masses = params['MassTable']

    parts = f['PartType%d'%parttype]
    mass = masses[parttype]*1.0e10
    x = np.array(parts['Coordinates'])
    kdt = KDTree( x )
    ngbs = kdt.query(x, k=npt, return_distance=True)
    rs = ngbs[0][:, -1]
    vol = 4.0/3.0*np.pi*rs*rs*rs
    dens = npt * mass / vol
    v = np.array(parts['Velocities'])
    dout = np.hstack((x, v, dens.reshape(dens.size, -1)))
    print('snapshot load %d particles, ranges:'%( snapshot, len(dout) ), end='')
    for i in range(3):
        print('[%f, %f]'%( dout[:, i].min(), dout[:, i].max() ), end='')
    print()
        
    if xselect:
        sel = np.ones( len(dout), dtype=bool )
        for i in range(3):
            sel = sel & ( dout[:, i] >= xselect[i][0] ) & ( dout[:, i] < xselect[i][1] )
        dout = dout[sel]
        
    with open( "%spart%d.npt%d.csv.%0.3d"%(outprefix,parttype,npt,snapshot), 'w' ) as fout:
        fout.write('x,y,z,vx,vy,vz,dens\n')
        for row in dout:
            fout.write( "%g,%g,%g,%g,%g,%g,%g\n"%tuple(row) )

In [107]:
fileprefix = 
for i in range(151):
    h5tocsv(i, 1, fileprefix, fileprefix, 10, [[-500., 500.], [-500., 500.], [-20., 20.]])
    h5tocsv(i, 2, fileprefix, fileprefix, 30)

load 40000 particles, ranges:[-191.413696, 192.293488][-133.081650, 131.808609][-99.407059, 99.054108]
load 20000 particles, ranges:[-147.736542, 114.755608][-94.760574, 84.449135][-56.419647, 50.155239]
load 40000 particles, ranges:[-190.564606, 190.864212][-133.018234, 131.765900][-99.028862, 99.031723]
load 20000 particles, ranges:[-146.801132, 113.137833][-95.532181, 83.789902][-55.748325, 50.589256]
load 40000 particles, ranges:[-189.671204, 189.390839][-132.910339, 131.732147][-98.732712, 98.966072]
load 20000 particles, ranges:[-145.770905, 111.488075][-96.243828, 83.024490][-55.009785, 50.937843]
load 40000 particles, ranges:[-188.732422, 187.872421][-132.756836, 131.652878][-98.542168, 98.855949]
load 20000 particles, ranges:[-144.642914, 109.805229][-96.893532, 82.153152][-54.204350, 51.198032]
load 40000 particles, ranges:[-187.748459, 186.309189][-132.557983, 131.528091][-98.306870, 98.701508]
load 20000 particles, ranges:[-143.417099, 108.089226][-97.481361, 81.178230][-53

In [40]:
f = h5py.File("./galaxy_collide/t2/output/snapshot_%0.3d.hdf5"%(0), 'r')
params = f['Header'].attrs
masses = params['MassTable']

parts = f['PartType2']
x = np.array(parts['Coordinates'])
v = np.array(parts['Velocities'])
xselect = [ [-250., 250.], [-250., 250.], [-2.5, 2.5] ]
sel = np.ones( len(x), dtype=bool )
for i in range(3):
    sel = sel & ( x[:, i] >= xselect[i][0] ) & ( x[:, i] < xselect[i][1] )
x = x[sel]
v = v[sel]

In [41]:
x.shape

(12667, 3)

In [37]:
nx, ny = 102, 102
xmin, ymin = -50.0, -50.0
xstep, ystep = 1., 1.
xid, yid = np.array((x[:, 0]-xmin)/xstep, dtype=int), np.array((x[:, 1]-ymin)/ystep, dtype=int)
ids = xid * ny + yid
ids[ (xid < 0) | (xid >= nx) | (yid < 0) | (yid >= ny) ]  = 0

vx = np.zeros( nx*ny, dtype=float )
vy = np.zeros( nx*ny, dtype=float )
ns = np.zeros( nx*ny, dtype=float ) + 1.0e-10
for i, id in enumerate(ids):
    vx[id] += v[i, 0]
    vy[id] += v[i, 1]
    ns[id] += 1.0
vx /= ns
vy /= ns
    
vx = vx.reshape((nx, ny))
vy = vy.reshape((nx, ny))

with open('./galaxy_collide/t2/output/part2.vgrid.csv.000', 'w') as f:
    f.write('x,y,z,vx,vy,vz\n')
    for iy in range(1,ny-1):
        for ix in range(1,nx-1):
            f.write( '%g,%g,%g,%g,%g,%g\n'%(xmin+ix*xstep, ymin+iy*ystep, 0., vx[ix,iy], vy[ix,iy], 0.) )

In [46]:
x.min()

-147.73654

In [42]:
vx[1:,1:].max()

310.15014645336

In [47]:
nx, ny, nz = 202, 202, 32
xmin, ymin, zmin = -200.0, -200.0, -15.0
xstep, ystep, zstep = 2., 2., 1.
xid, yid, zid = np.array((x[:, 0]-xmin)/xstep, dtype=int), np.array((x[:, 1]-ymin)/ystep, dtype=int), np.array((x[:, 2]-zmin)/zstep, dtype=int)
ids = (xid*ny + yid)*nz + zid
ids[ (xid < 0) | (xid >= nx) | (yid < 0) | (yid >= ny) | (zid < 0) | (zid >= nz) ]  = 0

vx = np.zeros( nx*ny*nz, dtype=float )
vy = np.zeros( nx*ny*nz, dtype=float )
vz = np.zeros( nx*ny*nz, dtype=float )
ns = np.zeros( nx*ny*nz, dtype=float ) + 1.0e-10
for i, id in enumerate(ids):
    vx[id] += v[i, 0]
    vy[id] += v[i, 1]
    vz[id] += v[i, 2]
    ns[id] += 1.0
vx /= ns
vy /= ns
vz /= ns
vx = vx.reshape((nx, ny, nz))
vy = vy.reshape((nx, ny, nz))
vz = vz.reshape((nx, ny, nz))

with open('./galaxy_collide/t2/output/part2.vgrid.csv.000', 'w') as f:
    f.write('x,y,z,vx,vy,vz\n')
    for iz in range(1,nz-1):
        for iy in range(1,ny-1):
            for ix in range(1,nx-1):
                f.write( '%g,%g,%g,%g,%g,%g\n'%(xmin+ix*xstep, ymin+iy*ystep, zmin+iz*zstep, vx[ix,iy,iz], vy[ix,iy,iz], vz[ix,iy,iz]) )